In [2]:
import ee

# Initialize the Earth Engine module.
ee.Initialize()

def print_s2_metadata(image_id):
    # Load a Sentinel-2 Image.
    image = ee.Image(image_id)

    # Get image metadata.
    info = image.getInfo()
    metadata = info['properties']

    # Print selected metadata.
    print("Acquisition Time:", metadata.get('system:time_start'))
    print("Cloud Coverage:", metadata.get('CLOUDY_PIXEL_PERCENTAGE'))
    print("Footprint:", metadata.get('system:footprint'))
    # Add more metadata fields as needed.

# Example usage
s2_image_id = 'COPERNICUS/S2/20190621T211041_20190621T211035_T12XVN'  # Replace with an actual image ID
print_s2_metadata(s2_image_id)


Acquisition Time: 1561151563000
Cloud Coverage: 88.7093
Footprint: {'type': 'LinearRing', 'coordinates': [[-110.5313630640149, 79.26894568372697], [-110.53142346061894, 79.26894644827712], [-115.80179921199189, 79.2323755771986], [-115.80200394115062, 79.23233573562166], [-115.8022340800813, 79.2323009580321], [-115.8022429657737, 79.23228600076543], [-115.59527422721567, 78.74212050367015], [-115.40568773002593, 78.25180987433377], [-115.4054935577514, 78.25177133142908], [-115.40532437288297, 78.25172806252048], [-115.40525077536289, 78.25172634529638], [-110.57028207701748, 78.28516248743838], [-110.57007658248824, 78.28519871647158], [-110.5698503417297, 78.28522952744252], [-110.56983570313358, 78.2852442759799], [-110.53094403950846, 79.26885573262079], [-110.53113884861223, 79.26889749392606], [-110.53130368785479, 79.2689434874317], [-110.5313630640149, 79.26894568372697]]}


In [8]:
import ee
from datetime import datetime, timedelta
import requests
import pandas as pd

ee.Initialize()

def get_s3_images_matching_s2(s2_footprint, s2_time, buffer_distance=100000):  # Buffer in meters
    # Define a time window (±2 hours from S2 image time)
    start_time = s2_time - timedelta(hours=2)
    end_time = s2_time + timedelta(hours=2)

    # Convert the footprint to an Earth Engine Geometry and buffer it
    s2_geometry = ee.Geometry.Polygon(s2_footprint['coordinates'])
    buffered_geometry = s2_geometry.buffer(buffer_distance)

    # Query for Sentinel-3 images within the time window and spatial extent
    s3_collection = ee.ImageCollection('COPERNICUS/S3/OLCI') \
        .filterDate(start_time, end_time) \
        .filterBounds(buffered_geometry)

    # Return the list of Sentinel-3 image IDs
    return s3_collection.aggregate_array('system:index').getInfo()

# Example usage
# Replace with your actual Sentinel-2 footprint and acquisition time
s2_footprint = {'type': 'LinearRing', 'coordinates': [[-110.5313630640149, 79.26894568372697], [-110.53142346061894, 79.26894644827712], [-115.80179921199189, 79.2323755771986], [-115.80200394115062, 79.23233573562166], [-115.8022340800813, 79.2323009580321], [-115.8022429657737, 79.23228600076543], [-115.59527422721567, 78.74212050367015], [-115.40568773002593, 78.25180987433377], [-115.4054935577514, 78.25177133142908], [-115.40532437288297, 78.25172806252048], [-115.40525077536289, 78.25172634529638], [-110.57028207701748, 78.28516248743838], [-110.57007658248824, 78.28519871647158], [-110.5698503417297, 78.28522952744252], [-110.56983570313358, 78.2852442759799], [-110.53094403950846, 79.26885573262079], [-110.53113884861223, 79.26889749392606], [-110.53130368785479, 79.2689434874317], [-110.5313630640149, 79.26894568372697]]}  # Sentinel-2 footprint
s2_time = datetime.strptime('2019-06-21T21:10:41', '%Y-%m-%dT%H:%M:%S')  # Sentinel-2 acquisition time

matched_s3_images = get_s3_images_matching_s2(s2_footprint, s2_time)
print("Matched Sentinel-3 Images:", matched_s3_images)


Matched Sentinel-3 Images: []


In [48]:
import ee
from datetime import datetime, timedelta
import numpy as np
ee.Initialize()

def get_s2_images_in_arctic(start_date, end_date,area, max_cloud_percentage=10):
    # Define the Arctic region bounding box
    arctic_region = ee.Geometry.Rectangle([-180, 60, 180, 90])

    # Filter the Sentinel-2 collection
    s2_collection = ee.ImageCollection('COPERNICUS/S2') \
        .filterDate(start_date, end_date) \
        .filterBounds(arctic_region) \
        .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', max_cloud_percentage))

    return s2_collection

def find_colocated_s3_for_s2(s2_image, buffer_distance=10000, time_window_hours=2):
    # Buffer the S2 footprint and define time window
    s2_geometry = s2_image.geometry().buffer(buffer_distance)
    s2_time = ee.Date(s2_image.get('system:time_start'))
    start_time = s2_time.advance(-time_window_hours, 'hour')
    end_time = s2_time.advance(time_window_hours, 'hour')

    # Query for S3 images
    s3_collection = ee.ImageCollection('COPERNICUS/S3/OLCI') \
        .filterDate(start_time, end_time) \
        .filterBounds(s2_geometry)

    return s3_collection.toList(s3_collection.size())

arctic_region_mask_line=np.loadtxt('region_mask_line.txt')
# Create a polygon object representing the Arctic Ocean boundary
region_mask_line_tt = [(lon, lat) for lon, lat in arctic_region_mask_line]
area_of_interest = ee.Geometry.LineString(coords=region_mask_line_tt, proj='EPSG:4326', geodesic=True)
# Example usage
start_date = '2019-03-01'
end_date = '2019-03-02'
s2_collection = get_s2_images_in_arctic(start_date, end_date,area_of_interest)

# Iterate over S2 images and find colocated S3 images
matched_pairs = []
s2_list = s2_collection.toList(s2_collection.size()).getInfo()
for s2_info in s2_list:
    s2_image = ee.Image(s2_info['id'])
    colocated_s3 = find_colocated_s3_for_s2(s2_image)
    s3_info_list = colocated_s3.getInfo()

    for s3_info in s3_info_list:
        matched_pairs.append((s2_info['id'], s3_info['id']))

# Print or process the matched pairs
for pair in matched_pairs:
    print("S2 Image:", pair[0], "has colocated S3 Image:", pair[1])


S2 Image: COPERNICUS/S2/20190301T235611_20190301T235610_T01WCM has colocated S3 Image: COPERNICUS/S3/OLCI/S3A_20190301T222350_20190301T222650
S2 Image: COPERNICUS/S2/20190301T235611_20190301T235610_T01WCM has colocated S3 Image: COPERNICUS/S3/OLCI/S3B_20190301T232521_20190301T232821
S2 Image: COPERNICUS/S2/20190301T235611_20190301T235610_T01WCS has colocated S3 Image: COPERNICUS/S3/OLCI/S3A_20190301T222350_20190301T222650
S2 Image: COPERNICUS/S2/20190301T235611_20190301T235610_T01WCS has colocated S3 Image: COPERNICUS/S3/OLCI/S3A_20190302T000449_20190302T000749
S2 Image: COPERNICUS/S2/20190301T235611_20190301T235610_T01WCS has colocated S3 Image: COPERNICUS/S3/OLCI/S3B_20190301T232521_20190301T232821
S2 Image: COPERNICUS/S2/20190301T235611_20190301T235610_T01WCT has colocated S3 Image: COPERNICUS/S3/OLCI/S3A_20190301T222350_20190301T222650
S2 Image: COPERNICUS/S2/20190301T235611_20190301T235610_T01WCT has colocated S3 Image: COPERNICUS/S3/OLCI/S3A_20190302T000449_20190302T000749
S2 Ima

In [18]:
print("S2 Image:", pair[0], "has colocated S3 Image:", pair[1])

S2 Image: COPERNICUS/S2/20190301T235611_20190301T235610_T60WWC has colocated S3 Image: COPERNICUS/S3/OLCI/S3B_20190301T232521_20190301T232821


In [35]:
import ee
from datetime import datetime, timedelta

ee.Initialize()

def get_s3_images_in_arctic(start_date, end_date, area):
    # Define the Arctic region bounding box
    arctic_region = ee.Geometry.Rectangle([-180, 60, 180, 90])

    # Filter the Sentinel-3 collection
    s3_collection = ee.ImageCollection('COPERNICUS/S3/OLCI') \
        .filterDate(start_date, end_date) \
        .filterBounds(arctic_region)

    return s3_collection

def find_colocated_s2_for_s3(s3_image, buffer_distance=5000, time_window_hours=2):
    # Buffer the S3 footprint and define time window
    s3_geometry = s3_image.geometry().buffer(buffer_distance)
    s3_time = ee.Date(s3_image.get('system:time_start'))
    start_time = s3_time.advance(-time_window_hours, 'hour')
    end_time = s3_time.advance(time_window_hours, 'hour')

    # Query for S2 images
    s2_collection = ee.ImageCollection('COPERNICUS/S2') \
        .filterDate(start_time, end_time) \
        .filterBounds(s3_geometry) \
        .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10))

    return s2_collection.toList(s2_collection.size())

# Example usage
arctic_region_mask_line = np.loadtxt('region_mask_line.txt')
region_mask_line_tt = [(lon, lat) for lon, lat in arctic_region_mask_line]
area_of_interest = ee.Geometry.LineString(coords=region_mask_line_tt, proj='EPSG:4326', geodesic=True)

start_date = '2019-03-01'
end_date = '2019-03-02'
s3_collection = get_s3_images_in_arctic(start_date, end_date, area_of_interest)

# Iterate over S3 images and find colocated S2 images
matched_pairs = []
s3_list = s3_collection.toList(s3_collection.size()).getInfo()
for s3_info in s3_list:
    s3_image = ee.Image(s3_info['id'])
    colocated_s2 = find_colocated_s2_for_s3(s3_image)
    s2_info_list = colocated_s2.getInfo()

    for s2_info in s2_info_list:
        matched_pairs.append((s3_info['id'], s2_info['id']))

# Print or process the matched pairs
for pair in matched_pairs:
    print("S3 Image:", pair[0], "has colocated S2 Image:", pair[1])


S3 Image: COPERNICUS/S3/OLCI/S3A_20190301T003100_20190301T003400 has colocated S2 Image: COPERNICUS/S2/20190301T002601_20190301T002602_T58VCP
S3 Image: COPERNICUS/S3/OLCI/S3A_20190301T003100_20190301T003400 has colocated S2 Image: COPERNICUS/S2/20190301T011649_20190301T011651_T56WNS
S3 Image: COPERNICUS/S3/OLCI/S3A_20190301T003100_20190301T003400 has colocated S2 Image: COPERNICUS/S2/20190301T011649_20190301T011651_T56WNT
S3 Image: COPERNICUS/S3/OLCI/S3A_20190301T003100_20190301T003400 has colocated S2 Image: COPERNICUS/S2/20190301T011649_20190301T011651_T56WPA
S3 Image: COPERNICUS/S3/OLCI/S3A_20190301T003100_20190301T003400 has colocated S2 Image: COPERNICUS/S2/20190301T011649_20190301T011651_T56WPB
S3 Image: COPERNICUS/S3/OLCI/S3A_20190301T003100_20190301T003400 has colocated S2 Image: COPERNICUS/S2/20190301T011649_20190301T011651_T56WPC
S3 Image: COPERNICUS/S3/OLCI/S3A_20190301T003100_20190301T003400 has colocated S2 Image: COPERNICUS/S2/20190301T011649_20190301T011651_T57WVR
S3 Ima

In [43]:
import ee
from datetime import datetime, timedelta

ee.Initialize()

def get_s3_images_in_arctic(start_date, end_date, area):
    # Define the Arctic region bounding box
    arctic_region = ee.Geometry.Rectangle([-180, 60, 180, 90])

    # Filter the Sentinel-3 collection
    s3_collection = ee.ImageCollection('COPERNICUS/S3/OLCI') \
        .filterDate(start_date, end_date) \
        .filterBounds(arctic_region)

    return s3_collection

def find_colocated_s2_for_s3(s3_image, area_of_interest, buffer_distance=100000, time_window_hours=2):
    # Buffer the S3 footprint and define time window
    arctic_region = ee.Geometry.Rectangle([-180, 60, 180, 90]).buffer(buffer_distance)
    s3_geometry = s3_image.geometry().buffer(buffer_distance)
    s3_time = ee.Date(s3_image.get('system:time_start'))
    start_time = s3_time.advance(-time_window_hours, 'hour')
    end_time = s3_time.advance(time_window_hours, 'hour')

    # Query for S2 images within the time window, spatial extent of S3, and within the Arctic region
    s2_collection = ee.ImageCollection('COPERNICUS/S2') \
        .filterDate(start_time, end_time) \
        .filterBounds(s3_geometry) \
        .filterBounds(arctic_region) \
        .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10))

    return s2_collection.toList(s2_collection.size())


# Example usage
arctic_region_mask_line = np.loadtxt('region_mask_line.txt')
region_mask_line_tt = [(lon, lat) for lon, lat in arctic_region_mask_line]
area_of_interest = ee.Geometry.LineString(coords=region_mask_line_tt, proj='EPSG:4326', geodesic=True)

start_date = '2019-03-01'
end_date = '2019-03-02'
s3_collection = get_s3_images_in_arctic(start_date, end_date, area_of_interest)

# Iterate over S3 images and find colocated S2 images
matched_pairs = []
s3_list = s3_collection.toList(s3_collection.size()).getInfo()
for s3_info in s3_list:
    s3_image = ee.Image(s3_info['id'])
    colocated_s2 = find_colocated_s2_for_s3(s3_image,area_of_interest)
    s2_info_list = colocated_s2.getInfo()

    for s2_info in s2_info_list:
        matched_pairs.append((s3_info['id'], s2_info['id']))

# Print or process the matched pairs
for pair in matched_pairs:
    print("S3 Image:", pair[0], "has colocated S2 Image:", pair[1])


EEException: Collection.toList: The value of 'count' must be positive. Got: 0.